

# 第10课：Actor-Critic方法

---

## Part 1：最简单的 Actor-Critic（QAC）

### 基本思想：

* **Actor**：负责策略 $\pi_\theta(a|s)$ 的参数化与更新
* **Critic**：负责估计动作值函数 $Q_w(s,a)$（或状态值函数 $V_w(s)$），为Actor提供策略梯度估计的基准

---

### QAC算法框架：

* Critic使用TD方法估计 $Q_w(s,a)$
* Actor基于Critic的估计，用梯度上升优化策略参数

---

### 关键更新：

* **Critic** 更新：

$$
w \leftarrow w + \beta \delta_t \nabla_w Q_w(s_t, a_t)
$$

其中，TD误差

$$
\delta_t = r_{t+1} + \gamma Q_w(s_{t+1}, a_{t+1}) - Q_w(s_t, a_t)
$$

* **Actor** 更新：

$$
\theta \leftarrow \theta + \alpha \delta_t \nabla_\theta \log \pi_\theta(a_t|s_t)
$$

---

### 解释：

* Critic评估策略的优劣，提供TD误差作为信号
* Actor利用TD误差调整策略，使策略向更好方向优化

---

## Part 2：Advantage Actor-Critic (A2C)

### 优化点：

用 **优势函数** 替代动作值函数，减少策略梯度的方差，提高学习稳定性。

---

### 计算优势函数：

$$
A(s,a) = Q(s,a) - V(s)
$$

* $Q(s,a)$：动作值估计
* $V(s)$：状态值估计

---

### A2C核心更新：

* Critic估计状态值 $V_w(s)$
* Actor更新利用优势函数：

$$
\theta \leftarrow \theta + \alpha A(s_t, a_t) \nabla_\theta \log \pi_\theta(a_t|s_t)
$$

* Critic的目标：

$$
\min_w \left( r_{t+1} + \gamma V_w(s_{t+1}) - V_w(s_t) \right)^2
$$

---

### 特点：

* 减少了方差，提升样本效率
* 广泛应用于深度强化学习框架

---

## Part 3：重要性采样和 Off-policy Actor-Critic

### 问题：

* Actor-Critic多为**on-policy**方法
* 离线或异策略数据无法直接利用

---

### 解决方案：重要性采样（Importance Sampling）

调整样本权重：

$$
w_t = \frac{\pi_\theta(a_t|s_t)}{\mu(a_t|s_t)}
$$

* $\mu$：行为策略
* $\pi_\theta$：目标策略

---

### Off-policy Actor-Critic方法：

* 利用重要性采样权重校正策略梯度
* 结合Replay Buffer，实现数据复用和稳定训练

---

## Part 4：Deterministic Actor-Critic (DPG)

### 背景：

* 传统策略梯度为随机策略
* 对连续动作空间，确定性策略能减少方差，提高效率

---

### 确定性策略：

$$
\mu_\theta(s) = a
$$

策略直接输出动作，不是概率分布。

---

### DPG算法核心：

* Critic估计 $Q(s,a)$
* Actor更新利用Critic的梯度：

$$
\nabla_\theta J(\theta) = \mathbb{E}_{s \sim \rho^\mu} \left[ \nabla_\theta \mu_\theta(s) \nabla_a Q^\mu(s,a) \big|_{a=\mu_\theta(s)} \right]
$$

---

### 特点：

* 适合高维连续动作空间
* 结合经验回放和目标网络实现稳定训练（即 DDPG 算法）

---

## 本课总结

* Actor-Critic方法有效结合策略优化与价值估计
* A2C用优势函数降低方差，提升性能
* 重要性采样支持离策略训练
* DPG针对连续动作提出确定性策略，提升效率


